### GOOGLE - GEMINI

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
from IPython.display import display, Markdown

load_dotenv(find_dotenv())

import google.generativeai as gemini_AI

GEMININAI_API_KEY = os.getenv("GEMININ_API_KEY")

gemini_AI.configure(api_key=GEMININAI_API_KEY)

model = gemini_AI.GenerativeModel('gemini-pro')

response = model.generate_content("Are you doing well?")

display(Markdown(response.text))


### OPENAI

In [ ]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
from IPython.display import display, Markdown
import asyncio
import openai

load_dotenv(find_dotenv())

client = OpenAI()


response = client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    response_format={"type": "text"},
    # response_format={"type": "json_object"},
    messages=[
        {
            "role": "system",
            # "content": "helpful assisstant that produces output in JSON",
            "content": "helpful assisstant",
        },
        {
            "role": "user",
            "content": "How are you?"
        },
    ],
)

# display(Markdown(response.choices[0].message))
response.choices[0].message